In [1]:
!pip install pandas numpy nltk emoji tqdm

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import json
import pandas as pd

# Đọc JSON
with open("data/2025-10-31/playlist_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Đưa về DataFrame cơ bản
df = pd.DataFrame(data)
print("✅ Dữ liệu gốc:", df.shape)
print(df.columns)


✅ Dữ liệu gốc: (49, 14)
Index(['video_id', 'title', 'description', 'publishedAt', 'channelId',
       'channelTitle', 'tags', 'viewCount', 'likeCount', 'commentCount',
       'duration', 'caption', 'transcript', 'comments'],
      dtype='object')


In [3]:
import re
import emoji
from tqdm import tqdm
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))  # Có thể tự thêm stopword tiếng Việt

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = emoji.replace_emoji(text, "")              # Xóa emoji
    text = re.sub(r"http\S+", "", text)               # Xóa link
    text = re.sub(r"[^0-9a-zA-ZÀ-ỹ\s]", " ", text)   # Xóa ký tự đặc biệt
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    words = [w for w in text.split() if w not in STOPWORDS]
    return " ".join(words)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\quocla\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [4]:
def flatten_comments(comment_list):
    all_comments = []
    if not isinstance(comment_list, list):
        return ""
    for c in comment_list:
        all_comments.append(c.get("text", ""))
        for r in c.get("replies", []):
            all_comments.append(r.get("text", ""))
    return " ".join(all_comments)


In [5]:
tqdm.pandas()

df["tags_text"] = df["tags"].apply(lambda x: " ".join(x) if isinstance(x, list) else "")
df["comments_text"] = df["comments"].progress_apply(flatten_comments)

# Làm sạch các trường text
for col in ["title", "description", "tags_text", "comments_text"]:
    df[col] = df[col].progress_apply(clean_text)

# Tạo cột tổng hợp nội dung text
df["combined_text"] = df["title"] + " " + df["description"] + " " + df["tags_text"] + " " + df["comments_text"]

print("✅ Hoàn tất xử lý dữ liệu.")
print(df[["video_id", "combined_text"]].head(3))


100%|██████████| 49/49 [00:00<00:00, 692.57it/s]


✅ Hoàn tất xử lý dữ liệu.
      video_id                                      combined_text
0  IM8lTK9Jbco  nghệ sĩ quyền linh chia sẻ về thuốc viên khớp ...
1  PEF-Fm_qlLM  mc quyền linh linh đã chọn nam dược bài thạch ...
2  62LO6w-x3mA  quyền linh giới thiệu thuốc chữa tiểu đường th...


In [6]:
output_path = "data/2025-10-31/playlist_data_clean.csv"
df.to_csv(output_path, index=False, encoding="utf-8")
print(f"💾 Đã lưu file sạch: {output_path}")


💾 Đã lưu file sạch: data/2025-10-31/playlist_data_clean.csv
